In [1]:
# load libs
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import matplotlib.pyplot as plt
import pickle
#from torch.utils.tensorboard import SummaryWriter

In [2]:
# definition of a simple fc model [11, 32, 64, 2]
class MOGANet(nn.Module):
    def __init__(self):
        super(MOGANet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(11, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 2),
        )
        
    def forward(self, x):
        x = self.fc(x)
        return x

In [3]:
random.seed(2019) 
torch.manual_seed(2019)
torch.cuda.manual_seed_all(2019)

model = MOGANet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5)

In [4]:
# load files
moga = pickle.load(open("moga.label.pkl", "rb"))
x_train, x_test = moga["x_train"], moga["x_test"]
y_train, y_test = moga["y_train"], moga["y_test"]

In [5]:
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train)
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test)

In [6]:
# set tensorboard directory
#writer = SummaryWriter('/home/ylu/moga/runs/1124')

In [7]:
for t in range(2000):
    train_acc = 0
    test_acc = 0
    
    model.train()
    Yp = model(x_train)
    loss = criterion(Yp, y_train)
    pred = Yp.max(1)[1] # get the index of the max log-probability
    train_acc = pred.eq(y_train).sum().item() / len(Yp)
    #print(train_acc)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    model.eval()
    with torch.no_grad():
        Yp_test = model(x_test)
        test_loss = criterion(Yp_test, y_test)
        pred = Yp_test.max(1)[1] # get the index of the max log-probability
        test_acc = pred.eq(y_test).sum().item() / len(Yp_test)
        #print(test_acc)
        scheduler.step(test_loss)
        
    #writer.add_scalars('Loss', {'train':train_acc,'test':test_acc}, t)

In [8]:
# save trained model
torch.save(model.state_dict(), 'moga-label-1124.pth')

In [9]:
model.eval()
with torch.no_grad():
    Yp = model(x_test)
    pred = Yp.max(1)[1] # get the index of the max log-probability
    print(pred.eq(y_test).sum().item() / len(Yp))

0.8155357142857143
